In [9]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [12]:
%%cython --cplus 
#cython: language_level=3, boundscheck=False, wraparound=False, initializedcheck=False, cdivision=True
# %%cython -a
# cython: profile=True
# cython: linetrace=True
# cython: binding=True
# distutils: define_macros=CYTHON_TRACE_NOGIL=1

cimport numpy as cnp
from cython.parallel import prange
import numpy as np
import pandas as pd
import time

cpdef target_mean_v3(data, y_name, x_name):
    cdef int x_max = data.shape[0]
    DTYPE = np.double

    cdef cnp.ndarray[double,ndim=1] result = np.zeros(x_max,dtype=np.float64)
    cdef cnp.ndarray[double,ndim=1] value_arr = np.zeros(10)
    cdef cnp.ndarray[double,ndim=1] count_arr = np.zeros(10)

    cdef cnp.ndarray[long] x_values =data[x_name].values
    cdef cnp.ndarray[long] y_values =data[y_name].values
    
    cdef int i = 0
    
    for i in prange(x_max,nogil=True):
        value_arr[x_values[i]]+=y_values[i]
        count_arr[x_values[i]]+=1
    for i in prange(x_max,nogil=True):
        
        result[i] = (value_arr[x_values[i]]-y_values[i])/( count_arr[x_values[i]]-1) 
    return result
        

    
  



In [13]:
y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])
start_time = time.time()
result = target_mean_v3(data, 'y', 'x')
print (result)
end_time = time.time()
time_cost = end_time - start_time
print('time_cost:',time_cost)


[0.50528541 0.51646091 0.51851852 ... 0.52539062 0.53413655 0.47188755]
time_cost: 0.002050638198852539


In [14]:
 target_mean_v3(data, 'y', 'x')

array([0.50528541, 0.51646091, 0.51851852, ..., 0.52539062, 0.53413655,
       0.47188755])